In [48]:
# pip install signalp-6-package/

In [49]:
# pip install tape-proteins

In [50]:
#HuggingFace Tranformers Package
# pip install transformers

In [51]:
# pip install keras

In [79]:
import numpy as np 
import matplotlib.pyplot as plt 
import os 
from scipy.stats import norm
import scipy.stats as sps
import cv2
from scipy.ndimage import convolve
import skimage
from scipy import ndimage
from scipy.spatial.distance import cdist
from nibabel.testing import data_path
import nibabel as nib
from sklearn.mixture import GaussianMixture
import os
import pandas as pd
import skimage.measure
import seaborn as sns
import scipy as sp
import gudhi as gd
import random
import sys
sys.path.append('../../') 
from Code.function_geometry import * 
from Code.codegeometry import * 
from PIL import Image
from sklearn.utils import shuffle

In [80]:
import sklearn.preprocessing   as skp
import sklearn.neighbors       as skn
import sklearn.model_selection as skm
import sklearn.decomposition   as skd
import sklearn.manifold        as skf
import sklearn.pipeline        as skl
import sklearn.svm             as sks
import sklearn.ensemble        as ske
import itertools
import tensorflow as tf

import gudhi.clustering.tomato as gdt
import gudhi.representations   as gdr

In [81]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
import torch
#tokenizer
from tape import ProteinBertModel, TAPETokenizer
# from tape import UniRepModel, UniRepTokenizer
# from tape import ProteinBertModel, ProteinBertTokenizer
from transformers import AutoTokenizer
from transformers import BertModel, BertTokenizer
# ProtGPT2
# from transformers import pipeline

In [83]:
from keras.models import Sequential
from keras.layers import Conv1D,Conv2D, Dense, Flatten, Dropout,MaxPooling2D
from keras import layers
from keras.layers import MaxPooling1D,MaxPooling2D
from keras.optimizers import SGD, RMSprop
from keras import regularizers

In [56]:
def readFastaFiles(filename):
    sequences = []
    labels = []
    with open(filename, 'r') as file:
        current_id = None
        current_seq = ''
        for line in file:
            if line.startswith('>'):
                if current_id and len(current_seq) > 0 and len(current_seq) <= 201:
                    labels.append(current_id[1:])
                    sequences.append(current_seq.replace('*', '').replace('\n', ''))
                current_id = line
                current_seq = ''
            else:
                current_seq += line
        # Add the last sequence
        if current_id and len(current_seq) > 0 and len(current_seq) <= 201:
            labels.append(current_id)
            sequences.append(current_seq.replace('*', '').replace('\n', ''))
    return labels,sequences

In [57]:
def createFaster(filename,labels,sequences):
    with open(filename, 'w') as f:
        # Write content to the file
        for i in range(len(sequences)):
        # for i in range(5):
            f.write(labels[i])
            f.write(sequences[i]+"\n")

In [58]:
# def resultSignalP(filename):
#     f=open(filename,'r')
#     result=[]
#     next(f)
#     next(f)
#     for line in f.readlines():
#         data = line.rstrip()  
#         index = data.__contains__('SP')
#         #Can we return the probabilty of SP and other
#         if index == True: 
#             split = data.split()
#             result.append(float(split[split.index("SP")+2]))
#         else:
#             result.append(0)
#     f.close()
#     return result  

In [59]:
#Creating a file of all signalP sequences
#getting supposed signalprotein
def signalProtein(filename,resultSignalp,sequences,labels):
    new_label = []
    new_sequence = []
    i = 0
    with open(filename, 'w') as f:
        for r in result_signalp:
            if r > 0 and r < 0.7:
                new_label.append(0)
            elif r > 0.7:
                new_label.append(1)
            if r > 0:
                f.write(labels[i]+"\n")
                f.write(sequences[i]+"\n")
                new_sequence.append(sequences[i])
            i+=1
    return new_label,new_sequence         

In [60]:
def resultSignalP(filename):
    f=open(filename,'r')
    result=[]
    next(f)
    next(f)
    for line in f.readlines():
        data = line.rstrip()  
        index = data.__contains__('SP')
        #Can we return the probabilty of SP and other
        if index == True: 
            result.append(1)
        else:
            result.append(0)
    f.close()
    return result  

In [61]:
def geotop_analysis(data):
    # U_train = U_train_b + U_train_m + U_test_b + U_test_m
    n = 100
    i = 0
    dgms_tda = []
    dgms_per_var = []
    dgms_per_linalg = []
    for U in data:
        print(i)
        L = np.linspace(np.min(U), np.max(U),  n)[::-1]
        #per = np.zeros(n)
        #area = np.zeros(n)
        #euler = np.zeros(n)
        #for t in range(len(L)):
         #   per[t] = Perimetre_Hermine(U, L[t])
          #  area[t] = Area(U, L[t])
           # euler[t] = Euler(U >= L[t])
        life, barecode, persistence, connected_comp, Per_total, Area_total, euler_total, per, area, euler = function_persistance(U, L, False)
        diagram_pondere_total_vari = np.zeros((len(life), 2))
        diagram_pondere_linalg = np.zeros((len(life), 2))
        diagram = np.zeros((len(life), 2))
        k = 0
        idx_keys = list(life.keys())
        for idx in idx_keys[1:]:
            diagram_pondere_total_vari[k][0] = (np.sum(per[idx]))*life[idx][1]
            diagram_pondere_total_vari[k][1] = (np.sum(per[idx]))*life[idx][0]
            diagram_pondere_linalg[k][0] = (np.linalg.norm(per[idx]))*life[idx][1]
            diagram_pondere_linalg[k][1] = (np.linalg.norm(per[idx]))*life[idx][0]
            diagram[0] = life[idx][1]
            diagram[1] = life[idx][0]
            k=k+1
        dgms_tda.append(diagram)
        dgms_per_linalg.append(diagram_pondere_linalg)
        dgms_per_var.append(diagram_pondere_total_vari)
        i=i+1
    return dgms_per_linalg

In [62]:
def concate_embedding_geotop(embedding,dgms,dim):
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest withe the remainning zeroes
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate((embedding[i], flat)), np.zeros(t))))
        i +=1
    return res

In [63]:
import tensorflow as tf
import keras

print("TensorFlow version:", tf.__version__)
print("Keras version:", keras.__version__)

TensorFlow version: 2.15.0
Keras version: 2.15.0


In [64]:
def concate_embedding_geotop(embedding,dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    long = embedding.shape[1]
    res = []
    i = 0
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - (long + flat.shape[0])
        res.append(np.concatenate((np.concatenate([embedding[i], flat]), np.zeros(t))))
        i +=1
    return res

In [65]:
def concat_embedding_geotop(dgms,dim):
    # len_data = max(array.shape[0] for array in dgms)
    res = []
    for array in dgms:
        flat = array.flatten()
        #flatten and concatenate the rest with zeroes
        t = dim - flat.shape[0]
        res.append(np.concatenate([flat, np.zeros(t)]))
    return res

In [66]:
# create a fasta file with the selected protein

In [67]:
# label,sequence=readFastaFiles("Arabidopsis_thaliana_new.fa")  # change to your own path

In [68]:
# len(sequence)

In [69]:
label,sequence=readFastaFiles("Araport11_pep_20220914")  # change to your own path

In [70]:
len(sequence)

10280

In [71]:
len(label)

10280

In [72]:
np.save("labels.npy",label)

## Data Manipulation

In [73]:
#Creating fasta file with protein <=200
# createFaster("arabidopsis_new.fasta",label,sequence)

### Embedding using TAPE

In [74]:
#Learning the representation of protein
model_tape = ProteinBertModel.from_pretrained('bert-base')
tokenizer_tape = TAPETokenizer(vocab='iupac')

In [75]:
# new_label,new_sequence

In [76]:
len(label)

10280

In [77]:
num_of_features = 768
embedded=np.zeros((len(label),num_of_features))
y=np.zeros(len(label))

i=0
for s in sequence:
    #tape
    print(i)
    token_ids = torch.tensor([tokenizer_tape.encode(s)])
    output = model_tape(token_ids)
    sequence_output = output[0]
    embedded[i]= np.array(np.mean(sequence_output.detach().numpy(),axis=1) )
    i=i+1

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175


KeyboardInterrupt: 

In [ ]:
np.save('arabidopsis_tape.npy', embedded)

In [ ]:
embedded.shape

(10280, 768)

In [ ]:
sequence[1262]

'MGIQTRSVLIIVAILTMMFSAHIAQSNSITMCVKHCAQNECLKAAKKPTPKICDEACKKICNNQLFGDEKWFVPAPKGSSRICRWAPKYCQF'

C:\Users\Kelly Larissa\AppData\Local\Temp\ipykernel_2408\2566444864.py:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:233.)
  token_ids = torch.tensor([tokenizer_tape.encode('MGLKVPHVICFVLWFSILVLLFQEFKFLKNNNKVIIYNDHDHHSLFGQRKTLLASNFDFTPFVKHVHHHHHHHHRDDVYEQPEAEPAGSHEIDQRYGVEKRLVPTGPNPLHH')])


In [ ]:
np.array(np.mean(sequence_output.detach().numpy(),axis=1) )


array([[-2.51333117e-01, -7.09935486e-01, -5.18622100e-01,
        -8.36126357e-02,  4.53375727e-01, -1.54175788e-01,
         1.68042615e-01,  7.33584687e-02, -5.15009165e-01,
         1.81494772e-01, -2.38725945e-01,  3.16049039e-01,
         2.28278786e-01,  7.87636280e-01, -2.00505927e-01,
         3.19484979e-01,  3.34603161e-01,  1.55769721e-01,
         1.05765522e-01,  2.70272434e-01,  1.59814596e+00,
        -9.70319867e-01, -1.26203358e-01,  3.02978545e-01,
         9.67091545e-02,  5.24719596e-01,  3.34913611e-01,
        -1.09978664e+00, -3.76599282e-01,  6.25169396e-01,
         5.20593762e-01, -5.09745955e-01,  4.36559588e-01,
        -2.35490352e-01,  8.68518353e-01, -2.50426400e-02,
        -3.89961183e-01, -2.89651930e-01, -2.05247700e-01,
        -3.11841428e-01,  7.83472955e-01, -1.66388556e-01,
        -6.56309202e-02,  9.83418822e-02, -1.01046383e+00,
        -1.23971604e-01,  9.31540504e-03, -8.19850385e-01,
         2.01578587e-01, -1.12365365e-01, -8.36353898e-0

In [ ]:
embedded[1262]

array([ 3.40335757e-01, -5.68999588e-01, -1.15445316e+00,  5.29415607e-01,
        5.97059354e-03, -2.06464812e-01,  2.62329578e-01, -7.57287741e-02,
        2.56462455e-01,  1.12809213e-02,  3.51638675e-01,  1.61379814e-01,
        9.13280904e-01,  3.47207248e-01,  1.66668713e-01,  6.52133703e-01,
        1.15961480e+00,  4.19961572e-01,  1.73337832e-01,  1.35348424e-01,
        5.77573776e-01, -1.20172262e+00,  1.80590898e-01, -7.20949844e-02,
        1.57171622e-01,  2.71871656e-01,  4.43178654e-01, -5.38368165e-01,
       -5.20531833e-02,  4.30627078e-01,  2.37819389e-01, -4.46539968e-01,
        6.10802829e-01, -1.22913647e+00,  8.32664490e-01, -1.70520012e-04,
       -2.04918664e-02, -1.88881591e-01, -9.98266757e-01, -3.68326187e-01,
        4.14292157e-01, -3.69497776e-01,  9.90944728e-02, -1.94944158e-01,
       -2.40582749e-01,  6.33064806e-01, -3.55774581e-01, -4.53895122e-01,
        7.45002091e-01,  5.36838591e-01, -3.61379057e-01,  1.02892406e-01,
       -1.77922934e-01, -

In [ ]:

import numpy as np
# Save the array to a file
# np.save('arabidopsis_tape.npy', embedded)

# # Later, to reload the array
embedded = np.load('arabidopsis_tape.npy')

In [ ]:
embedded

array([[ 0.03735537, -0.54995698,  0.03819402, ...,  0.30786389,
        -0.17370194, -1.05217326],
       [ 0.31556338, -0.54165047, -0.23029067, ...,  0.25627723,
        -0.51904792, -1.16927266],
       [ 0.26519731, -0.27949029, -0.42844746, ..., -0.44679058,
         0.00260051, -0.2349245 ],
       ...,
       [ 0.57984281, -0.36114416, -0.27833793, ...,  0.56995946,
        -0.37486482, -0.77060574],
       [ 0.37213781, -0.73763239, -0.25906518, ...,  0.5051235 ,
         0.22619204, -0.91627204],
       [ 0.45445824, -0.3206242 , -0.36179981, ...,  0.40787935,
        -0.20276164, -1.25479293]])

In [ ]:
len(label)

10280

In [ ]:
embedded.shape

(10280, 768)

In [ ]:
embedded.shape[1]

768

In [ ]:
embedded_3d = embedded.reshape(embedded.shape[0],embedded.shape[1],1)

In [ ]:
embedded_3d.shape

(10280, 768, 1)

### Classification

In [ ]:
from gtda.homology import VietorisRipsPersistence

# Track connected components, loops, and voids
homology_dimensions = [0]

# Collapse edges to speed up H2 persistence calculation!
persistence = VietorisRipsPersistence(
    metric="euclidean",
    homology_dimensions=homology_dimensions,
    n_jobs=6,
    collapse_edges=True,
)

persistence_diagrams = persistence.fit_transform(embedded_3d)

KeyboardInterrupt: 

In [ ]:
np.save("persitence_data.npy",persistence_diagrams)

In [ ]:
persistence_diagrams = np.load("persitence_data.npy")

In [85]:
from gtda.plotting import plot_diagram
from scipy.stats import wasserstein_distance

# Circle
# plot_diagram(persistence_diagrams[3])

In [ ]:
persistence_diagrams[10][:,1]

array([1.54972076e-06, 1.59442425e-06, 5.03659248e-06, 1.07586384e-05,
       1.18911266e-05, 1.32322311e-05, 1.82911754e-05, 2.48551369e-05,
       2.51531601e-05, 2.56299973e-05, 2.60919333e-05, 3.01897526e-05,
       3.73423100e-05, 4.38988209e-05, 4.41372395e-05, 4.51505184e-05,
       4.80627641e-05, 4.83393669e-05, 4.84585762e-05, 4.85181808e-05,
       5.99026680e-05, 6.40451908e-05, 6.70999289e-05, 6.81281090e-05,
       7.09295273e-05, 7.09444284e-05, 7.12275505e-05, 7.14659691e-05,
       7.23004341e-05, 7.40885735e-05, 7.69495964e-05, 8.30665231e-05,
       8.83042812e-05, 1.06036663e-04, 1.08897686e-04, 1.16229057e-04,
       1.17000658e-04, 1.17629766e-04, 1.19507313e-04, 1.21161342e-04,
       1.23769045e-04, 1.24543905e-04, 1.28626823e-04, 1.28775835e-04,
       1.29818916e-04, 1.31294131e-04, 1.33633614e-04, 1.36896968e-04,
       1.36971474e-04, 1.37478113e-04, 1.38476491e-04, 1.38938427e-04,
       1.39355659e-04, 1.39832497e-04, 1.40434131e-04, 1.42633915e-04,
      

In [ ]:
len(persistence_diagrams[0])

767

In [ ]:
np.save('persistence_diagrams.npy', persistence_diagrams)

NameError: name 'persistence_diagrams' is not defined

In [ ]:
np.save('persistence_data.npy', persistence_diagrams)

In [ ]:
# Compute the Wasserstein distance between the two diagrams
distance = wasserstein_distance(persistence_diagrams[0][:,1], persistence_diagrams[0][:,1])

print("Wasserstein distance:", distance)


Wasserstein distance: 0.0


In [ ]:
length = len(persistence_diagrams)

In [ ]:
sequence[1262]

'MGIQTRSVLIIVAILTMMFSAHIAQSNSITMCVKHCAQNECLKAAKKPTPKICDEACKKICNNQLFGDEKWFVPAPKGSSRICRWAPKYCQF'

In [ ]:
persistence_diagrams[1262]

array([[0.00000000e+00, 6.85453415e-07, 0.00000000e+00],
       [0.00000000e+00, 1.99675560e-06, 0.00000000e+00],
       [0.00000000e+00, 2.05636024e-06, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.03186989e-01, 0.00000000e+00],
       [0.00000000e+00, 2.73682594e-01, 0.00000000e+00],
       [0.00000000e+00, 6.91490507e+00, 0.00000000e+00]])

In [86]:
for i in range(len(persistence_diagrams)):
    for j in range(len(persistence_diagrams)):
        wessertein_dist[i,j] = wasserstein_distance(persistence_diagrams[i][:,1], persistence_diagrams[j][:,1])

NameError: name 'wessertein_dist' is not defined

In [ ]:
length

10280

In [ ]:
wessertein_dist = np.zeros(shape=(length, length))

In [ ]:
persistence_diagrams[0]

array([[0.00000000e+00, 5.06639481e-06, 0.00000000e+00],
       [0.00000000e+00, 5.45382500e-06, 0.00000000e+00],
       [0.00000000e+00, 7.83801079e-06, 0.00000000e+00],
       ...,
       [0.00000000e+00, 2.79704452e-01, 0.00000000e+00],
       [0.00000000e+00, 3.19888592e-01, 0.00000000e+00],
       [0.00000000e+00, 7.55657578e+00, 0.00000000e+00]])

In [110]:
token_ids = torch.tensor([tokenizer_tape.encode('MKTNRNRPINILIVFFLLTTARAATRNWTNRTHRTVPKVQHAYYAYPHRSCESFSRPYARSMCIELERIHRSSRQPLFSPPPPPTEIDQRYGVEKRLVPSGPNPLHN')])
output = model_tape(token_ids)
sequence_output = output[0]
res = np.array(np.mean(sequence_output.detach().numpy(),axis=1) )
per = persistence.fit_transform(res.reshape(1,embedded.shape[1],1))

In [90]:
per[0][:,1]

array([7.74860382e-06, 9.71555710e-06, 1.55866146e-05, 1.88350677e-05,
       2.15172768e-05, 2.18003988e-05, 2.27540731e-05, 2.36034393e-05,
       2.41100788e-05, 2.43782997e-05, 2.55294144e-05, 3.29911709e-05,
       3.89516354e-05, 4.17865813e-05, 4.35796101e-05, 4.78047878e-05,
       5.01275063e-05, 5.47468662e-05, 6.15864992e-05, 6.43432140e-05,
       6.44028187e-05, 6.50361180e-05, 6.66230917e-05, 6.98864460e-05,
       8.22395086e-05, 9.00626183e-05, 9.79825854e-05, 9.91523266e-05,
       1.02594495e-04, 1.04248524e-04, 1.04427338e-04, 1.10447407e-04,
       1.12771988e-04, 1.18136406e-04, 1.21474266e-04, 1.28477812e-04,
       1.37150288e-04, 1.43587589e-04, 1.51737593e-04, 1.54025853e-04,
       1.62839890e-04, 1.68979168e-04, 1.73166394e-04, 1.77592039e-04,
       1.80602074e-04, 1.80907547e-04, 1.86964869e-04, 1.89185143e-04,
       1.92880630e-04, 1.93953514e-04, 1.94743276e-04, 1.96874142e-04,
       2.01514456e-04, 2.03073025e-04, 2.03095376e-04, 2.05770135e-04,
      

In [111]:
res = [wasserstein_distance(persistence_diagrams[i][:,1], per[0][:,1]) for i in range(len(persistence_diagrams))]

In [115]:
res.sort()

In [116]:
res

[0.0,
 0.0003035006875589971,
 0.0003035006875589971,
 0.0003635276995679397,
 0.0003741683602039193,
 0.000377099946319736,
 0.00038983555030407737,
 0.0003900206329075152,
 0.0003949497987132504,
 0.000397460232215608,
 0.00040045307768263273,
 0.0004058233776458823,
 0.0004097196140546333,
 0.00040995182505610756,
 0.0004139436395824253,
 0.0004166757267191934,
 0.00041719568125913236,
 0.00041762017402093807,
 0.0004177515002178829,
 0.0004211823869058855,
 0.00042217417657764014,
 0.0004257107796475271,
 0.0004276292634741819,
 0.00042846112175403707,
 0.0004301625780513508,
 0.000430763924963082,
 0.00043307239890360747,
 0.00043307239890360747,
 0.00043370628822351836,
 0.00043411055067372594,
 0.00043411055067372594,
 0.00043411055067372594,
 0.00043411055067372594,
 0.0004434267195115271,
 0.0004496259833843872,
 0.00045359144114793447,
 0.00045554198457033,
 0.00045885876014784894,
 0.0004591550325884984,
 0.0004620996125970114,
 0.00046509196568945705,
 0.0004693884781204828

In [112]:
sorted_indices = sorted(range(len(res)), key=lambda x: res[x])

In [114]:
label[2184]

'AT1G69320.1 | Symbols: CLE10 | CLAVATA3/ESR-RELATED 10 | chr1:26061925-26062248 FORWARD LENGTH=107\n'

In [113]:
sorted_indices

[2184,
 6607,
 6608,
 6665,
 3937,
 1488,
 2847,
 8332,
 385,
 3380,
 1840,
 6689,
 3261,
 4624,
 8694,
 9886,
 7251,
 1293,
 9377,
 5709,
 9418,
 10147,
 3274,
 7143,
 9285,
 8622,
 3902,
 3903,
 4219,
 9641,
 9642,
 9643,
 9644,
 10077,
 4480,
 5436,
 4260,
 8926,
 1675,
 10065,
 450,
 2297,
 7285,
 10050,
 9347,
 2644,
 384,
 8717,
 2081,
 2082,
 1555,
 3828,
 9726,
 2269,
 9305,
 9751,
 9907,
 5057,
 5058,
 5059,
 2583,
 7184,
 3508,
 7589,
 4551,
 9404,
 633,
 3411,
 3126,
 4954,
 9385,
 4830,
 4294,
 3412,
 2715,
 402,
 6890,
 3015,
 5140,
 1823,
 8579,
 9084,
 6584,
 2116,
 9829,
 5072,
 1781,
 8896,
 2459,
 2301,
 3062,
 6198,
 6199,
 2618,
 5391,
 8005,
 6316,
 3277,
 6783,
 8778,
 9750,
 5014,
 5940,
 9037,
 2718,
 119,
 8908,
 3885,
 4924,
 3540,
 1489,
 1142,
 10059,
 2005,
 8419,
 6417,
 9348,
 2623,
 5346,
 5211,
 9469,
 7288,
 6481,
 3954,
 3955,
 2170,
 6985,
 242,
 9555,
 6070,
 8018,
 4496,
 1098,
 1622,
 7318,
 1780,
 8413,
 8414,
 1233,
 9040,
 4255,
 4414,
 2286,
 

In [91]:
for i in range(len(persistence_diagrams)):
    result = wasserstein_distance(persistence_diagrams[i][:,1], per[0][:,1])
    # if result == 0:
    print(result,i)

0.002693997036178111 0
0.004523388455552608 1
0.001989140285930619 2
0.001989140285930619 3
0.0017463667310635709 4
0.001989140285930619 5
0.005107207325920919 6
0.003979128998910007 7
0.004579861835335795 8
0.004579861835335795 9
0.0035498272088772312 10
0.0035498272088772312 11
0.0035498272088772312 12
0.00245355875483278 13
0.003364026925511025 14
0.003930436356498212 15
0.004466250456105771 16
0.0028536258291329943 17
0.0056501579293500425 18
0.0013295293114308545 19
0.0013295293114308545 20
0.0013295293114308545 21
0.0013295293114308545 22
0.004215213357236513 23
0.003593633460449166 24
0.003723524619345206 25
0.004627159777986148 26
0.001663272090728532 27
0.001663272090728532 28
0.0007364054620593796 29
0.0007151690404847045 30
0.002265429538064469 31
0.002265429538064469 32
0.002471006756030336 33
0.006800614249047919 34
0.0064390270543955045 35
0.002795178103130116 36
0.0020673062199740994 37
0.004105392195687449 38
0.0035081873728311404 39
0.0010947735710439956 40
0.005837808

In [ ]:
wasserstein_distance(persistence_diagrams[0][:,1],persistence_diagrams[0][:,1])

0.0

In [ ]:
wessertein_dist.shape

(10280, 10280)

In [ ]:
len(label)

10280

In [ ]:
df = pd.DataFrame(wessertein_dist, index=label, columns=label)

In [ ]:
df

,AT1G01020.2 | Symbols: ARV1 | | chr1:7315-8666 REVERSE LENGTH=191\n,AT1G01020.6 | Symbols: ARV1 | | chr1:7315-8419 REVERSE LENGTH=104\n,"AT1G01100.1 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n","AT1G01100.2 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n","AT1G01100.3 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=96\n","AT1G01100.4 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",AT1G01130.1 | Symbols: no symbol available | no full name available | chr1:61905-63811 REVERSE LENGTH=124\n,AT1G01160.1 | Symbols: GIF2 | GRF1-interacting factor 2 | chr1:72583-73883 FORWARD LENGTH=195\n,AT1G01170.1 | Symbols: no symbol available | no full name available | chr1:74105-74443 REVERSE LENGTH=83\n,AT1G01170.2 | Symbols: no symbol available | no full name available | chr1:74105-74443 REVERSE LENGTH=83\n,...,ATMG00070.1 | Symbols: NAD9 | NADH dehydrogenase subunit 9 | chrm:24665-25237 FORWARD LENGTH=190\n,ATMG00080.1 | Symbols: RPL16 | ribosomal protein L16 | chrm:23285-23824 FORWARD LENGTH=179\n,ATMG00210.1 | Symbols: RPL5 | ribosomal protein L5 | chrm:184762-185319 REVERSE LENGTH=185\n,"ATMG00480.1 | Symbols: ATP8, ORFB | ATP SYNTHASE 8 | chrm:247016-247492 REVERSE LENGTH=158\n",ATMG00640.1 | Symbols: ORF25 | | chrm:362206-362784 FORWARD LENGTH=192\n,ATMG00980.1 | Symbols: RPSL2 | | chrm:288446-288823 FORWARD LENGTH=125\n,ATMG00990.1 | Symbols: NAD3 | NADH dehydrogenase 3 | chrm:288044-288400 FORWARD LENGTH=118\n,ATMG01080.1 | Symbols: ATP9 | mitochondrial F0-ATPase subunit 9 | chrm:270961-271185 REVERSE LENGTH=74\n,ATMG01270.1 | Symbols: RPS7 | mitochondrial ribosomal protein S7 | chrm:79559-80005 FORWARD LENGTH=148\n,"ATMG01275.1 | Symbols: ND1, NAD1, NAD1C, NAD1A | NADH DEHYDROGENASE 1, NADH dehydrogenase 1A, NADH dehydrogenase 1C, NAD(P)H DEHYDROGENASE 1 | chrm:58315-59481 FORWARD LENGTH=90\n"
AT1G01020.2 | Symbols: ARV1 | | chr1:7315-8666 REVERSE LENGTH=191\n,0.000000,0.002341,0.004177,0.004177,0.002867,0.004177,0.003282,0.006169,0.002265,0.002265,...,0.002429,0.003655,0.002427,0.001466,0.002080,0.003497,0.001379,0.000711,0.004055,0.001154
AT1G01020.6 | Symbols: ARV1 | | chr1:7315-8419 REVERSE LENGTH=104\n,0.002341,0.000000,0.005682,0.005682,0.004185,0.005682,0.001044,0.007646,0.000654,0.000654,...,0.000475,0.001551,0.000858,0.001155,0.000957,0.001724,0.001875,0.002058,0.002221,0.002937
"AT1G01100.1 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",0.004177,0.005682,0.000000,0.000000,0.002294,0.000000,0.006351,0.003700,0.005692,0.005692,...,0.005975,0.006996,0.006069,0.004850,0.005593,0.006989,0.004748,0.004317,0.007330,0.004736
"AT1G01100.2 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=112\n",0.004177,0.005682,0.000000,0.000000,0.002294,0.000000,0.006351,0.003700,0.005692,0.005692,...,0.005975,0.006996,0.006069,0.004850,0.005593,0.006989,0.004748,0.004317,0.007330,0.004736
"AT1G01100.3 | Symbols: RPP1A, RPP1.1 | 60S acidic ribosomal protein P1-1, RPP1 co-orthologous gene 1 | chr1:50284-50954 REVERSE LENGTH=96\n",0.002867,0.004185,0.002294,0.002294,0.000000,0.002294,0.004750,0.005542,0.004505,0.004505,...,0.004627,0.005569,0.004814,0.003575,0.004459,0.005693,0.003569,0.002848,0.005468,0.003333
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ATMG00980.1 | Symbols: RPSL2 | | chrm:288446-288823 FORWARD LENGTH=125\n,0.003497,0.001724,0.006989,0.006989,0.005693,0.006989,0.001605,0.008705,0.001554,0.001554,...,0.001628,0.000821,0.001534,0.002345,0.001897,0.000000,0.003047,0.003319,0.001342,0.004269
ATMG00990.1 | Symbols: NAD3 | 

In [ ]:
np.save("wesserstein_distance.npy", wessertein_dist)

In [ ]:
df.to_csv('wassertein.csv', index=False)

In [ ]:
column_names = df.columns

In [ ]:
np.save("column_names.npy", column_names)